<a href="https://colab.research.google.com/github/Lee-Gunju/AI-paper-code-review-for-personal-project/blob/master/PixelCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tqdm import tqdm

In [ ]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)
n_residual_blocks = 5

# The data, split between train and test sets
(x, _), (y, _) = keras.datasets.mnist.load_data()

# Concatenate all of the images together
data = np.concatenate((x, y), axis=0)

# Round all pixel values less than 33% of the max 256 value to 0
# anything above this value gets rounded up to 1 so that all values are either
# 0 or 1

data = np.where(data < (0.33 * 256), 0, 1)
data = data.astype(np.float32)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
# The first layer is the PixelCNN layer. This layer simply
# builds on the 2D convolutional layer, but includes masking.

class PixelConvLayer(layers.Layer):
  def __init__(self, mask_type, **kwargs):
    super(PixelConvLayer, self).__init__()
    self.mask_type = mask_type 
    self.conv = layers.Conv2D(**kwargs)

  def build(self, input_shape):
    # Build the conv2d layer to initialize kernel variables
    self.conv.build(input_shape)
    # Use the initialized kernel to create the mask
    kernel_shape = self.conv.kernel.get_shape()
    self.mask = np.zeros(shape = kernel_shape)
    self.mask[: kernel_shape[0] // 2, ...] = 1.0 
    self.mask[kernel_shape[0] // 2, : kernel_shape[1] //2, ...] = 1.0
    if self.mask_type == "B":
      self.mask[kernel_shape[0] //2, kernel_shape[1] //2, ...] = 1.0 
  
  def call(self, inputs):
    self.conv.kernel.assign(self.conv.kernel * self.mask)
    return self.conv(inputs)


# Next, we build our residual block layer.
# This is just a normal residual block, but based on the PixelConvLayer.
class ResidualBlock(keras.layers.Layer):
  def __init__(self, filters, **kwargs):
    super(ResidualBlock, self).__init__(**kwargs)
    self.conv1 = keras.layers.Conv2D(filters = filters, kernel_size= 1, activation='relu')
    self.pixel_conv = PixelConvLayer(mask_type = 'B', filters = filters // 2, kernel_size=  3, activation='relu', padding = 'same')
    self.conv2 = keras.layers.Conv2D(filters = filters, kernel_size = 1, activation='relu')
  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.pixel_conv(x)
    x = self.conv2(x)
    return keras.layers.add([inputs, x])

In [ ]:
inputs = keras.Input(shape=input_shape)
x = PixelConvLayer(mask_type = 'A', filters = 128, kernel_size = 7, activation = 'relu', padding = 'same')(inputs)

for _ in range(n_residual_blocks):
  x = ResidualBlock(filters = 128)(x)

for _ in range(2):
  x = PixelConvLayer(mask_type='B', filters = 128, kernel_size = 1, strides = 1, activation = 'relu', padding ='same')(x)

out = keras.layers.Conv2D(
    filters=1, kernel_size=1, strides=1, activation="sigmoid", padding="valid"
)(x)

pixel_cnn = keras.Model(inputs, out)
adam = keras.optimizers.Adam(learning_rate=0.0005)
pixel_cnn.compile(optimizer=adam, loss="binary_crossentropy")

pixel_cnn.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
pixel_conv_layer_3 (PixelCon (None, 28, 28, 128)       6400      
_________________________________________________________________
residual_block_1 (ResidualBl (None, 28, 28, 128)       98624     
_________________________________________________________________
residual_block_2 (ResidualBl (None, 28, 28, 128)       98624     
_________________________________________________________________
residual_block_3 (ResidualBl (None, 28, 28, 128)       98624     
_________________________________________________________________
residual_block_4 (ResidualBl (None, 28, 28, 128)       98624     
_________________________________________________________________
residual_block_5 (ResidualBl (None, 28, 28, 128)       98624 

In [ ]:
pixel_cnn.fit(
    x=data, y=data, batch_size=128, epochs=50, validation_split=0.1, verbose=2
)

Epoch 1/50
493/493 - 64s - loss: 0.1142 - val_loss: 0.0939
Epoch 2/50
493/493 - 32s - loss: 0.0913 - val_loss: 0.0898
Epoch 3/50
493/493 - 32s - loss: 0.0893 - val_loss: 0.0884
Epoch 4/50
493/493 - 32s - loss: 0.0881 - val_loss: 0.0876
Epoch 5/50
493/493 - 32s - loss: 0.0874 - val_loss: 0.0872
Epoch 6/50
493/493 - 32s - loss: 0.0868 - val_loss: 0.0864
Epoch 7/50
493/493 - 32s - loss: 0.0863 - val_loss: 0.0861
Epoch 8/50
493/493 - 32s - loss: 0.0859 - val_loss: 0.0862
Epoch 9/50
493/493 - 32s - loss: 0.0856 - val_loss: 0.0863
Epoch 10/50
493/493 - 32s - loss: 0.0853 - val_loss: 0.0854
Epoch 11/50
493/493 - 32s - loss: 0.0850 - val_loss: 0.0852
Epoch 12/50
493/493 - 32s - loss: 0.0847 - val_loss: 0.0856
Epoch 13/50
493/493 - 32s - loss: 0.0846 - val_loss: 0.0856
Epoch 14/50
493/493 - 32s - loss: 0.0844 - val_loss: 0.0852
Epoch 15/50
493/493 - 32s - loss: 0.0842 - val_loss: 0.0843
Epoch 16/50
493/493 - 32s - loss: 0.0840 - val_loss: 0.0845
Epoch 17/50
493/493 - 32s - loss: 0.0839 - val_lo

In [ ]:
from IPython.display import Image, display

# Create an empty array of pixels.
batch = 4
pixels = np.zeros(shape = (batch, ) + (pixel_cnn.input_shape)[1:])
batch, rows, cols, channels = pixels.shape 

# Iterate over the pixels because generation has to be done sequentially pixel by pixel.
for row in tqdm(range(rows)):
  for col in range(cols):
    for channel in range(channels):
      # Feed the whole array and retrieving the pixel value probabilities for the next
      # pixel
      probs = pixel_cnn.predict(pixels)[:, row, col, channel]
      # Use the probabilities to pick pixel values and append the values to the image
      # frame.
      pixels[:, row, col, channel] = tf.math.ceil(probs - tf.random.uniform(probs.shape))


def deprocess_image(x):
  # Stack the single channeled black and white image to RGB values.
  x = np.stack((x,x,x), 2)

  x *= 255.0
  # Convert to uint8 and clip to the valid range [0, 255]
  x = np.clip(x, 0, 255).astype("uint8")
  return x

# Iterate over the generated images and plot them with matplotlib.
for i, pic in enumerate(pixels):
  keras.preprocessing.image.save_img("generated_image_{}.png".format(i), deprocess_image(np.squeeze(pic, -1)))


display(Image("generated_image_0.png"))
display(Image("generated_image_1.png"))
display(Image("generated_image_2.png"))
display(Image("generated_image_3.png"))

100%|██████████| 28/28 [00:32<00:00,  1.15s/it]
